In [226]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import timeit
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,BatchNormalization, Activation
from tensorflow.keras.activations import relu
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mean_absolute_error

In [ ]:
# TODO: set up correct dateRange and weight
# TODO: normalize model inputs

In [3]:
train1  = pd.read_csv('train_2.csv')
missing = train1.isnull()
nonMissingIndex = np.logical_not(missing).values
train1 = train1.fillna(0)
train1.iloc[:,1:] = train1.iloc[:,1:].astype(np.float32)
n = train1.shape[0]

In [4]:
a = train1.Page.str.split('mediawiki.org|wikimedia.org|wikipedia.org',expand=True)
b = np.where(a.loc[:,2].isnull(),a.loc[:,1],a.loc[:,2])
_,index_array = np.unique(b,return_inverse=True) # access & agent
website_array = [train1.Page.str.contains(web).values.astype(np.int8) \
                 for web in ['mediawiki.org','wikimedia.org','wikipedia.org']] # website
lang_array = train1.Page.str.extract('([a-z]{2}).wikipedia').fillna('nan').values.flatten()
_,lang_array = np.unique(lang_array,return_inverse=True) # language

In [5]:
Page_X = \
np.concatenate([OneHotEncoder(dtype=np.float32,sparse=False).fit_transform(lang_array[:,np.newaxis]),\
                OneHotEncoder(dtype=np.float32,sparse=False).fit_transform(index_array[:,np.newaxis]),\
                np.stack(website_array,1)],1)

In [34]:
date_index=pd.to_datetime(train1.columns[1:]).to_series().reset_index(drop=True)
def extract_date_info(timestamp,mss):
    return np.array([mss, timestamp.day, \
                     timestamp.dayofyear]+\
                    [1 if i==timestamp.year else 0 for i in [2015, 2016, 2017]]+\
                    [1 if i==timestamp.weekday() else 0 for i in range(7)]+\
                    [1 if i==timestamp.month else 0 for i in range(12)])

In [35]:
d = extract_date_info(date_index[3],3).shape[0]

In [36]:
max_ = lambda df,start,interval: np.max(df.iloc[:,start-interval:start].values,1)
min_ = lambda df,start,interval: np.min(df.iloc[:,start-interval:start].values,1)
std_ = lambda df,start,interval: np.std(df.iloc[:,start-interval:start].values,1)
mean_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start].values,1)
growth_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start],1) -\
                                    np.mean(df.iloc[:,start-2*interval:start-interval],1)

fun_list = [max_,min_,std_,mean_,growth_]
interval_list = [14,30,60]
T = train1.shape[1]

In [171]:
def Generator(batchSize,dateRange,weight):
    while True:
        j = np.random.choice(range(*dateRange),p=weight)
        i = np.random.choice(range(j+2,j+64))
        index_ = np.random.choice(np.where(nonMissingIndex[:,i])[0],size=batchSize)
        temp = train1.iloc[index_]
        Lookback_X = np.stack([fun(temp,j,interval) for fun in fun_list for interval in interval_list],1)
        yield ([np.concatenate([Page_X[index_],\
                                Lookback_X,\
                                np.broadcast_to(extract_date_info(date_index[i-1],i-j),(batchSize,d)),\
                                temp.iloc[:,j-1:j].values,\
                                temp.iloc[:,i-90:i-89].values,\
                                temp.iloc[:,i-120:i-119].values,\
                                temp.iloc[:,i-180:i-179].values,\
                                temp.iloc[:,i-365:i-364].values],1).astype(np.float32),\
               temp.iloc[:,j-1].values[:np.newaxis]],\
               temp.iloc[:,i].values[:np.newaxis])

In [37]:
downSampleRate = 0.25
j = T - 64
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
testData = []
testMargain = []
for i in range(j+2,j+64):
    temp = np.concatenate([train1.iloc[:,i].values[:,np.newaxis],\
                           Page_X,\
                           np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,d)),\
                           Lookback_X,\
                           train1.iloc[:,j-1:j].values,\
                           train1.iloc[:,i-90:i-89].values,\
                           train1.iloc[:,i-120:i-119].values,\
                           train1.iloc[:,i-180:i-179].values,\
                           train1.iloc[:,i-365:i-364].values],1).astype(np.float32)
    nonmissing_index = np.logical_not(missing.iloc[:,i].values)
    random_index = np.random.rand(nonmissing_index.sum())<downSampleRate
    testData.append(temp[nonmissing_index][random_index])
    testMargain.append(train1.iloc[:,j-1].values[nonmissing_index][random_index])
testData = np.concatenate(testData,0)
testMargain = np.concatenate(testMargain,0)    

In [165]:
testY = testData[:,0][:np.newaxis]
testX = testData[:,1:]
testMargain = testMargain[:,np.newaxis]
del testData

In [244]:
batchSize = 1000
depth = 5
d = testX.shape[1]
L2 = 1e-2
learningR = 1e-1
eps = 0.1

In [245]:
X_tf = tf.placeholder(tf.float32,(None,d))
base_tf = tf.placeholder(tf.float32,shape=(None,1))
y_tf = tf.placeholder(tf.float32,(None,1))

In [246]:
model = keras.Sequential()

In [247]:
for _ in range(depth):   
    model.add(Dense(d,kernel_regularizer=regularizers.l2(L2)))
    model.add(BatchNormalization())
    model.add(Activation(relu))

In [248]:
model.add(Dense(1,activation=relu,kernel_regularizer=regularizers.l2(L2)))

In [249]:
yhat_tf = model(X_tf) + base_tf

In [236]:
cost1 = mean_absolute_error(y_tf,yhat_tf)

In [240]:
train_op1 = tf.train.AdamOptimizer(learning_rate=learningR).minimize(cost1)

In [ ]:
cost2 = tf.abs(y_tf-yhat_tf)/(tf.abs(y_tf)+tf.abs(yhat_tf)+eps)

In [ ]:
train_op2 = tf.train.AdamOptimizer(learning_rate=learningR).minimize(cost2)

Make Submission

In [ ]:
date_index = date_index.append(pd.Series(pd.date_range('2017-09-11','2017-11-13')))

In [ ]:
date_index = date_index.reset_index(drop=True)

In [ ]:
j = T
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
testData = []
testMargain = []
mapping_index = []
for i in range(j+2,j+64):
    temp = np.concatenate([Page_X,\
                           np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,d)),\
                           Lookback_X,\
                           train1.iloc[:,j-1:j].values,\
                           train1.iloc[:,i-90:i-89].values,\
                           train1.iloc[:,i-120:i-119].values,\
                           train1.iloc[:,i-180:i-179].values,\
                           train1.iloc[:,i-365:i-364].values],1).astype(np.float32)

    testData.append(temp)
    testMargain.append(train1.iloc[:,j-1].values)
    mapping_index.append(train1.iloc[:,0].str.cat([str(date_index[i-1])[:10]]*n,'_'))
    
testData = np.concatenate(testData,0)
testMargain = np.concatenate(testMargain,0)
mapping_index = np.concatenate(mapping_index,0)

In [ ]:
# Data corruption when creating a a DMatrix with label being a non-contiguous ndarray #2554
dtest = xgb.DMatrix(np.ascontiguousarray(testData))
dtest.set_base_margin(np.ascontiguousarray(testMargain))

In [ ]:
yhat = np.maximum(model_gbm.predict(dtest,output_margin=True),0)

In [ ]:
yhat = pd.DataFrame({'Page':mapping_index,'Visits':yhat})

In [ ]:
key = pd.read_csv('key_2.csv')

In [ ]:
key = key.merge(yhat,on='Page')

In [ ]:
key.drop('Page',axis=1,inplace=True)

In [ ]:
key.to_csv('Submissions/gbm1.csv',index=False)